In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision.models import efficientnet_b0
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
from torchvision.datasets import CIFAR10
import time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
train_dataset = CIFAR10(root="./data", train=True,
                        download=True, transform=transform)
test_dataset = CIFAR10(root="./data", train=False,
                        download=True, transform=transform)
train_data_size = len(train_dataset)
test_data_size = len(test_dataset)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
def train_and_valid(model, loss_function, optimizer, epochs=25):
    history = list()
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))

        model.train()

        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0

        for inputs, labels in tqdm(train_dataloader,total=len(train_dataloader)):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs,labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            ret, predictions = torch.max(outputs.data,1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            train_acc += acc.item() * inputs.size(0)

        with torch.no_grad():
            model.eval()

            for inputs, labels in tqdm(test_dataloader,total=len(test_dataloader)):
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = loss_function(outputs, labels)

                test_loss += loss.item() * inputs.size(0)
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))
                acc = torch.mean(correct_counts.type(torch.FloatTensor))
                test_acc += acc.item() * inputs.size(0)

        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size
        avg_test_loss = test_loss/test_data_size
        avg_test_acc = test_acc/test_data_size

        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])

        if best_acc < avg_test_acc:
            best_acc = avg_test_acc
            best_epoch = epoch + 1
            torch.save(model.state_dict(), 'weights/best_model.pth')
            

        epoch_end = time.time()

        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\Test: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_test_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))

    return model, history


In [5]:
model = efficientnet_b0(pretrained=True)
model.classifier.linear = nn.Linear(1000, 10)
model.to(device)
num_epochs = 30
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
trained_model, history = train_and_valid(model, loss_func, optimizer, num_epochs)

c:\Users\Zhang\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\Zhang\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1/30


100%|██████████| 313/313 [00:16<00:00, 19.30it/s]


Epoch: 001, Training: Loss: 0.2970, Accuracy: 83.5560%, 
	\Test: Loss: 0.2970, Accuracy: 89.9700%, Time: 414.5447s
Best Accuracy for validation : 0.8997 at epoch 001
Epoch: 2/30


100%|██████████| 313/313 [00:17<00:00, 18.17it/s]


...

Epoch: 012, Training: Loss: 0.2992, Accuracy: 96.9760%, 
	\Test: Loss: 0.2992, Accuracy: 92.6000%, Time: 682.0001s
Best Accuracy for validation : 0.9354 at epoch 009
Epoch: 13/30


100%|██████████| 313/313 [00:35<00:00,  8.78it/s]


Epoch: 013, Training: Loss: 0.2493, Accuracy: 97.2720%, 
	\Test: Loss: 0.2493, Accuracy: 93.4000%, Time: 648.0194s
Best Accuracy for validation : 0.9354 at epoch 009
Epoch: 14/30


 73%|███████▎  | 1140/1563 [07:29<02:46,  2.54it/s]


KeyboardInterrupt: 